## Inputs
---

In [1]:
# reading data and working with arrays
import os
import zarr, nrrd
import numpy as np

# data paths
p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-unittest'
fix_path = p + '/fix.n5'
mov_path = p + '/mov.n5'
exp_factor = 1

out_p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-python'

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr.attrs.asdict()
fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * [1, 2, 2]
fix_spacing_s2 = fix_spacing * [2, 4, 4]
fix_spacing_s3 = fix_spacing * [4, 8, 8]
fix_spacing_s4 = fix_spacing * [8, 16, 16]
fix_spacing_s5 = fix_spacing * [16, 32, 32]
print(f'fix_spacing: {fix_spacing}')
print(f'fix_spacing_s1: {fix_spacing_s1}')
print(f'fix_spacing_s2: {fix_spacing_s2}')
print(f'fix_spacing_s3: {fix_spacing_s3}')
print(f'fix_spacing_s4: {fix_spacing_s4}')
print(f'fix_spacing_s5: {fix_spacing_s5}')

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr.attrs.asdict()
mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * [1, 2, 2]
mov_spacing_s2 = mov_spacing * [2, 4, 4]
mov_spacing_s3 = mov_spacing * [4, 8, 8]
mov_spacing_s4 = mov_spacing * [8, 16, 16]
mov_spacing_s5 = mov_spacing * [16, 32, 32]

print(f'mov_spacing: {mov_spacing}')
print(f'mov_spacing_s1: {mov_spacing_s1}')
print(f'mov_spacing_s2: {mov_spacing_s2}')
print(f'mov_spacing_s3: {mov_spacing_s3}')
print(f'mov_spacing_s4: {mov_spacing_s4}')
print(f'mov_spacing_s5: {mov_spacing_s5}')

global_scale = 's5'
fix_affine_spacing = fix_spacing_s5
mov_affine_spacing = mov_spacing_s5

local_scale = 's4'
fix_deform_spacing = fix_spacing_s4
mov_deform_spacing = mov_spacing_s4

fix_spacing: [0.42 0.23 0.23]
fix_spacing_s1: [0.42 0.46 0.46]
fix_spacing_s2: [0.84 0.92 0.92]
fix_spacing_s3: [1.68 1.84 1.84]
fix_spacing_s4: [3.36 3.68 3.68]
fix_spacing_s5: [6.72 7.36 7.36]
mov_spacing: [0.42 0.23 0.23]
mov_spacing_s1: [0.42 0.46 0.46]
mov_spacing_s2: [0.84 0.92 0.92]
mov_spacing_s3: [1.68 1.84 1.84]
mov_spacing_s4: [3.36 3.68 3.68]
mov_spacing_s5: [6.72 7.36 7.36]


In [ ]:
nrrd.write(out_p+'/fix.nrrd', fix_zarr[f'/c3/{global_scale}'][...].transpose(2,1,0), compression_level=2)
nrrd.write(out_p+'/mov.nrrd', mov_zarr[f'/c3/{global_scale}'][...].transpose(2,1,0), compression_level=2)

## Alignment
---

### Compute Affine

In [2]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# get global alignment channels
fix = fix_zarr[f'/c3/{global_scale}'][...]
mov = mov_zarr[f'/c3/{global_scale}'][...]

# define alignment steps
affine_kwargs = {
    'initial_condition': 'CENTER',
    'alignment_spacing':4.0, # increase this to throw out data
    'shrink_factors':[2],
    'smooth_sigmas':[2],
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':100,
    },
}

steps = [('affine', affine_kwargs)]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_affine_spacing, mov_affine_spacing,
    steps,
)

/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'array.rechunk-threshold' has been deprecated; please use 'array.rechunk.threshold' instead
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'dataframe.shuffle-compression' has been deprecated; please use 'dataframe.shuffle.compression' instead
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.events-log-length' has been deprecated;

Wed Jun  5 13:48:44 2024 Run affine -> {'initial_condition': 'CENTER', 'alignment_spacing': 4.0, 'shrink_factors': [2], 'smooth_sigmas': [2], 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 100}}
Wed Jun  5 13:48:44 2024 All affine args: {'initial_condition': 'CENTER', 'alignment_spacing': 4.0, 'shrink_factors': [2], 'smooth_sigmas': [2], 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 100}}
Wed Jun  5 13:48:44 2024 Affine align -> {'shrink_factors': [2], 'smooth_sigmas': [2], 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 100}}
Wed Jun  5 13:48:44 2024 Configure IRM args: {'shrink_factors': [2], 'smooth_sigmas': [2], 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 100}}
Wed Jun  5 13:48:44 2024 Configure IRM Metric: MMI -> {} Optimizer: RSGD -> {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 100} Shrink factors: [2] Smooth sigmas: [2]
LEVEL:  0  ITE

### Apply affine

In [3]:
# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s3, mov_spacing_s3,
    transform_list=[affine,],
)


In [4]:
# Save affine
os.makedirs(out_p, exist_ok=True)
np.savetxt(out_p+'/affine.mat', affine)

In [6]:
# write affine transformed vol
os.makedirs(out_p, exist_ok=True)
nrrd.write(out_p+'/affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

In [ ]:
# load precomputed results
affine = np.loadtxt(out_p+'/affine.mat')

### Prepare cluster args

In [17]:
cluster_kwargs={
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
        'distributed.comm.retry.count':10,
        'distributed.comm.timeouts.connect':'600s',
        'distributed.scheduler.worker-saturation': 1,
        'distributed.scheduler.unknown-task-duration':'60m',
    },
}

### Calculate deform transformation

In [18]:
%%time
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline

# FASTER

# get full resolution data
fix = fix_zarr[f'/c3/{local_scale}']
mov = mov_zarr[f'/c3/{local_scale}']

# define alignment steps
affine_kwargs = {
    'smooth_sigmas':(0.25,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 1.0,
}

deform_kwargs = {
    'smooth_sigmas':(0.25,),
    'control_point_spacing':50.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 2.0,
}

steps = [ ('affine', affine_kwargs,), ('deform', deform_kwargs,), ]

# deform
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_deform_spacing, mov_deform_spacing,
    steps=steps,
    blocksize=[128, 128, 128],
    static_transform_list=[affine,],
    write_path=out_p+'/deform.zarr',
    cluster_kwargs=cluster_kwargs,
)

Block index:  (0, 0, 2) 
Slices:  (slice(0, 120, None), slice(0, 192, None), slice(192, 343, None))
Block index:  (0, 0, 0) 
Slices:  (slice(0, 120, None), slice(0, 192, None), slice(0, 192, None))
Block index:  (0, 2, 1) 
Slices:  (slice(0, 120, None), slice(192, 448, None), slice(64, 320, None))
Block index:  (0, 1, 2) 
Slices:  (slice(0, 120, None), slice(64, 320, None), slice(192, 343, None))
Block index:  (0, 1, 1) 
Slices:  (slice(0, 120, None), slice(64, 320, None), slice(64, 320, None))
Block index:  (0, 3, 0) 
Slices:  (slice(0, 120, None), slice(320, 566, None), slice(0, 192, None))
Block index:  (0, 1, 0) 
Slices:  (slice(0, 120, None), slice(64, 320, None), slice(0, 192, None))
Block index:  (0, 0, 1) 
Slices:  (slice(0, 120, None), slice(0, 192, None), slice(64, 320, None))
Block index:  (0, 2, 0) 
Slices:  (slice(0, 120, None), slice(192, 448, None), slice(0, 192, None))
Block index:  (0, 2, 2) 
Slices:  (slice(0, 120, None), slice(192, 448, None), slice(192, 343, None))


2024-06-05 13:57:37,170 - distributed.worker.memory - WARNING - Worker is at 90% memory usage. Pausing worker.  Process memory: 11.54 GiB -- Worker memory limit: 12.80 GiB
2024-06-05 13:57:37,180 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 11.54 GiB -- Worker memory limit: 12.80 GiB
2024-06-05 13:57:37,285 - distributed.nanny.memory - WARNING - Worker tcp://10.101.10.21:59363 (pid=36982) exceeded 95% memory budget. Restarting...
2024-06-05 13:57:37,546 - distributed.nanny - WARNING - Restarting worker


Block index:  (0, 3, 1) 
Slices:  (slice(0, 120, None), slice(320, 566, None), slice(64, 320, None))
Wed Jun  5 14:06:15 2024 Run affine -> {'smooth_sigmas': (0.25,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 25}, 'alignment_spacing': 1.0}
Wed Jun  5 14:06:15 2024 All affine args: {'smooth_sigmas': (0.25,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 25}, 'alignment_spacing': 1.0}
Wed Jun  5 14:06:15 2024 Affine align -> {'smooth_sigmas': (0.25,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 25}}
Wed Jun  5 14:06:15 2024 Configure IRM args: {'smooth_sigmas': (0.25,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 25}}
Wed Jun  5 14:06:15 2024 Configure IRM Metric: MMI -> {} Optimizer: RSGD -> {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 25} Shrink factors: (1,) Smooth sigmas: (0.25,)
LEVEL:  0  ITERATION:  0  METRIC:  -0.0893870415574

2024-06-05 14:06:27,312 - tornado.application - ERROR - Uncaught exception GET /status/ws (::1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/tornado/web.py", line 3301, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expir

LEVEL:  0  ITERATION:  5  METRIC:  -0.09808417303143606
LEVEL:  0  ITERATION:  6  METRIC:  -0.10004368576396279
LEVEL:  0  ITERATION:  7  METRIC:  -0.1019915190726478
LEVEL:  0  ITERATION:  8  METRIC:  -0.10388656357895594
LEVEL:  0  ITERATION:  9  METRIC:  -0.10573245237379407
LEVEL:  0  ITERATION:  10  METRIC:  -0.10756595890819329
LEVEL:  0  ITERATION:  11  METRIC:  -0.1093804171198158
LEVEL:  0  ITERATION:  12  METRIC:  -0.1111904936476173
LEVEL:  0  ITERATION:  13  METRIC:  -0.11305975279614963
LEVEL:  0  ITERATION:  14  METRIC:  -0.11503212097007899
LEVEL:  0  ITERATION:  15  METRIC:  -0.11702607493672519
LEVEL:  0  ITERATION:  16  METRIC:  -0.11896022499059423
LEVEL:  0  ITERATION:  17  METRIC:  -0.12077418706215762
LEVEL:  0  ITERATION:  18  METRIC:  -0.12245281790080771
LEVEL:  0  ITERATION:  19  METRIC:  -0.12398083413032664
LEVEL:  0  ITERATION:  20  METRIC:  -0.1253753196245114
LEVEL:  0  ITERATION:  21  METRIC:  -0.1266023247112558
LEVEL:  0  ITERATION:  22  METRIC:  -0.12

Task exception was never retrieved
future: <Task finished name='Task-479' coro=<Client._gather.<locals>.wait() done, defined at /opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-484' coro=<Client._gather.<locals>.wait() done, defined at /opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task

TimeoutError: 

In [ ]:
# read already saved result
deform = zarr.open(out_p+'/deform.zarr', mode='r')

### Apply deformation

In [ ]:
from bigstream.piecewise_transform import distributed_apply_transform

blocksize = (64,)*3
channels = ['c3']

# s3 channels
fix = fix_zarr[f'/c3/{local_scale}']
for channel in channels:
    mov = mov_zarr[f'/{channel}/{local_scale}']
    aligned = distributed_apply_transform(
        fix, mov, fix_spacing_s3, mov_spacing_s3,
        transform_list=[affine, deform,],
        transform_spacing=((1,1,1), fix_spacing*fix_deform_spacing),
        blocksize=blocksize,
        cluster_kwargs=cluster_kwargs
    )
    nrrd.write(out_p+f'/r2{channel}{local_scale}_aligned_to_r1.nrrd', aligned.transpose(2,1,0), compression_level=2)


Compare deform output for nextflow and jupyter runs

In [ ]:
deform_zarr_name='/nrs/scicompsoft/rokicki/flyefish/output_bigstream/deform.zarr'
kr_deform_n5_name='/nrs/scicompsoft/rokicki/flyefish/registration/r1-r2/deform.n5'
kr_inv_deform_n5_name='/nrs/scicompsoft/rokicki/flyefish/registration/r1-r2/inv-deform.n5'
cg_deform_n5_name='/nrs/scicompsoft/goinac/multifish/mark/registration/r1-r2/deform.n5'
cg_inv_deform_n5_name='/nrs/scicompsoft/goinac/multifish/mark/registration/r1-r2/inv-deform.n5'

deform_zarr = zarr.open(deform_zarr_name, mode='r')
deform_n5_container = zarr.open(store=zarr.N5FSStore(cg_deform_n5_name), mode='r')
deform_n5 = deform_n5_container['c3/s2']
inv_deform_n5_container = zarr.open(store=zarr.N5FSStore(cg_inv_deform_n5_name))
inv_deform_n5 = inv_deform_n5_container['c3/s2']


In [ ]:
deform_zarr[0:10,0, 0]

In [ ]:
deform_n5[200:210,0, 0]

In [ ]:
inv_deform_n5[300,1200,1100]